FETCH HISTORIC DATA FROM YFINANCE

In [ ]:
import os
import pandas as pd
import yfinance as yf
from datetime import date

start_date = "2020-01-01"
end_date = date.today().strftime("%Y-%m-%d")  # ve formátu YYYY-MM-DD

# Načtení seznamu tickerů ze souboru CSV
df = pd.read_csv("../datasets/tickers.csv")
symbols = df["symbol"].tolist()

data = pd.DataFrame()

for symbol in symbols:
    try:
        ticker_data = yf.download(symbol, start=start_date, end=end_date, auto_adjust=True)
        
        ticker_data.columns = ticker_data.columns.get_level_values(0)
        ticker_data = ticker_data.reset_index()
        ticker_data = ticker_data.rename(columns={"price": "id"})
        ticker_data['symbol'] = symbol
        
        data = pd.concat([data, ticker_data], ignore_index=True)
    except Exception as e:
            print(f"Fetch error {symbol}: {e}")

print(data)

In [ ]:
data.to_csv('../datasets/yahoo_historic_data.csv',index=False)

HISTORIC DATASET REVIEW

In [1]:
import pandas as pd

df = pd.read_csv('../datasets/yahoo_historic_data.csv')

In [2]:
#remove stocks that have less than 1000 days of history
pocet_radku_alt = df["symbol"].value_counts()

symbols_to_keep = pocet_radku_alt[pocet_radku_alt >= 1000].index
df = df[df["symbol"].isin(symbols_to_keep)]

In [3]:
unique_symbols = df["symbol"].nunique()
print("Počet unikátních symbolů:", unique_symbols)

rows_on_date = df[df["Date"] == "2025-03-14"].shape[0]
print("Počet řádků s datem 2025-03-14:", rows_on_date)

#rows with empty values
print(df.isnull().sum())

#move the 'symbol' column into the first position
cols = df.columns.tolist()
cols.remove('symbol')
new_order = ['symbol'] + cols
df = df[new_order]
df

Počet unikátních symbolů: 478
Počet řádků s datem 2025-03-14: 478
Date      0
Close     0
High      0
Low       0
Open      0
Volume    0
symbol    0
dtype: int64


,symbol,Date,Close,High,Low,Open,Volume
0,AA,2020-01-02,20.655470,21.079766,20.554218,21.079766,3062500
1,AA,2020-01-03,20.732613,20.838687,20.395105,20.414391,3011800
2,AA,2020-01-06,20.250460,20.684399,20.154029,20.578324,3060400
3,AA,2020-01-07,20.559034,20.776004,20.240813,20.308314,4986900
4,AA,2020-01-08,19.729734,20.626541,19.671876,20.539753,3706300
...,...,...,...,...,...,...,...
648941,ZS,2025-03-10,194.690002,204.600006,192.740005,203.470001,3057300
648942,ZS,2025-03-11,199.360001,204.414993,195.199997,196.404999,2431700
648943,ZS,2025-03-12,197.789993,203.970001,195.160004,202.940002,2217000
648944,ZS,2025-03-13,189.580002,196.494995,186.669998,196.179993,2491900


In [ ]:
#stocks that have at least 1000 days of history

symbols = df['symbol'].unique()

df_symbols = pd.DataFrame(symbols, columns=['symbol'])

df_symbols.to_csv('../datasets/new_tickers_list.csv', index=False)

FETCH COMPANY INFORMATION DATA FROM YFINANCE

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import date

start_date = "2020-01-01"
end_date = date.today().strftime("%Y-%m-%d")  # ve formátu YYYY-MM-DD

# Načtení seznamu tickerů ze souboru CSV
df = pd.read_csv("../datasets/new_tickers_list.csv")
symbols = df["symbol"].tolist()

data = pd.DataFrame()

for symbol in symbols:
    try:
        stock = yf.Ticker(symbol)
        info = stock.info

        company_data = {
                "symbol": symbol,
                "short_name": info.get("shortName"),
                "region": info.get("region"),
                "exchange": info.get("exchange"),
                "exchange_timezone": info.get("exchangeTimezoneShortName"),
                "market_cap": info.get("marketCap"),
                "beta": info.get("beta"),
        }
        new_df = pd.DataFrame([company_data])
        
        data = pd.concat([data, new_df], ignore_index=True)
    except Exception as e:
            print(f"Fetch error {symbol}: {e}")

print(data)

In [ ]:
data.to_csv('../datasets/yahoo_company_info.csv',index=False)

COMPANY INFO DATASET REVIEW

In [1]:
import pandas as pd

df = pd.read_csv('../datasets/yahoo_company_info.csv')

In [2]:
df.head()

,symbol,short_name,region,exchange,exchange_timezone,market_cap,beta
0,AA,Alcoa Corporation,US,NYQ,EDT,8680380416,2.553
1,AAL,"American Airlines Group, Inc.",US,NMS,EDT,7147851264,1.270
2,AAP,Advance Auto Parts Inc.,US,NYQ,EDT,2247615232,1.293
3,AAPL,Apple Inc.,US,NMS,EDT,3207068385280,1.178
4,ABBV,AbbVie Inc.,US,NYQ,EDT,373848178688,0.598


In [3]:
print(df.isnull().sum())

symbol               0
short_name           0
region               0
exchange             0
exchange_timezone    0
market_cap           0
beta                 2
dtype: int64
